# Ridge Regression Model

In [173]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder
import seaborn as sns

In [174]:
colsToLoad =["INSTNM","CONTROL","ADM_RATE","ADM_RATE_ALL","ACTCMMID","ACTENMID","ACTMTMID","ACTWRMID","SAT_AVG","SAT_AVG_ALL","SATMTMID","UGDS","HIGHDEG",  "TUITFTE", "INEXPFTE",  "COSTT4_P",
             "COSTT4_A", "PCTFLOAN","COMP_ORIG_YR2_RT", "UGDS_WHITE","UGDS_BLACK","UGDS_HISP","UGDS_ASIAN","UGDS_AIAN","UGDS_NHPI","UGDS_2MOR","UGDS_NRA","UGDS_UNKN","PPTUG_EF","COSTT4_A","COSTT4_P","TUITIONFEE_IN","TUITIONFEE_OUT","TUITIONFEE_PROG","TUITFTE","INEXPFTE","AVGFACSAL","PCTPELL","DEATH_YR3_RT","COMP_ORIG_YR3_RT","LOAN_DEATH_YR3_RT","LOAN_COMP_ORIG_YR3_RT","DEATH_YR4_RT","COMP_ORIG_YR4_RT","COMPL_RPY_1YR_RT","AGE_ENTRY","COUNT_NWNE_P10","COUNT_WNE_P10","MN_EARN_WNE_P10","MD_EARN_WNE_P10","COMPL_RPY_1YR_RT"]
scoreCardDF = pd.read_csv("MERGED2013_14_PP.csv", index_col="INSTNM",usecols=colsToLoad)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (1606) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### drop all rows that do not have a value for target variable

In [175]:
scoreCardDF =scoreCardDF[(scoreCardDF["COMPL_RPY_1YR_RT"] != "PrivacySuppressed") & (scoreCardDF["COMPL_RPY_1YR_RT"] != np.nan)  & (scoreCardDF["COMPL_RPY_1YR_RT"] != None) &(scoreCardDF["COMPL_RPY_1YR_RT"].notna())]

### The following part shows the initial input features preparation
    1. Drop the un-needed columns features (Index and target Cols)
    2. Divide the features into two parts
        2.1 Categorial features
        2.2 float features

In [176]:
# all features without the target & 
input_features =  [f for f in colsToLoad if f not in ["INSTNM","COMPL_RPY_1YR_RT"]] 
cat_columns = ["HIGHDEG","CONTROL"]
float_columns = [f for f in input_features if f not in cat_columns]
target_feature = "COMPL_RPY_1YR_RT"

## Data Pre-processing:
    1. Replacing Privacy Suppressed value with null
    2. One-Hot-Encoder for categorial features
    3. Replace null valeues of input dataset with the "Median"

In [177]:
# this function is designed to convert PrivacySuppressed value in any input column to null.
def cleanPrivacySuppressed(dataFrame):
    for colName in dataFrame.columns:
        dataFrame[colName] =  dataFrame[colName].replace("PrivacySuppressed", np.nan)
        dataFrame[colName] = dataFrame[colName].astype(float)
    return dataFrame 

In [178]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
class PrivacySuppressedHandler(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  
    def transform(self, X, y=None):
        return cleanPrivacySuppressed(X)

In [179]:
# pipe line for all float columns
# it is a separate pipe line, because there is a potential to add more preprocessing steps based on model evaluation
float_pipeline = ColumnTransformer([
         ("privacy_suppressed_hnadler", PrivacySuppressedHandler(),float_columns),        
    ])

# pipe line for all Category columns
cat_pipeline = ColumnTransformer([
         ("hot_encoder",OneHotEncoder(sparse=False),cat_columns)
    ])

In [180]:
from sklearn.pipeline import FeatureUnion
full_pipeline = FeatureUnion(
    transformer_list=[ 
        ("float_pipeline", float_pipeline),
        ("cat_pipeline", cat_pipeline)
])

In [181]:
processed_input_features = full_pipeline.fit_transform(scoreCardDF)
target_output = scoreCardDF[target_feature]

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [182]:
processed_input_features = pd.DataFrame(processed_input_features)

### Splitting the data to training set (80%) and testing set (20%)

In [183]:
#--split data to test and training data
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(processed_input_features, target_output.values, test_size=0.2, random_state=0)

### Using SciKit Simple Imputer to replace null values with median value

In [184]:
from sklearn.impute import SimpleImputer

def replaceNulls(dataFrame):
    imputer = SimpleImputer(strategy='median')
    scoreCardDF_imputed = pd.DataFrame(imputer.fit_transform(dataFrame))
    scoreCardDF_imputed.columns = dataFrame.columns
    return scoreCardDF_imputed, imputer

In [185]:
train_X,imp = replaceNulls(train_X)
test_X = imp.transform(test_X)

In [186]:
catHotColumns =["HIGHDEG_Non-degree-granting","HIGHDEG_Certificate_degree","HIGHDEG_Associate_degree","HIGHDEG_Associate_degree","HIGHDEG_Associate_degree","CONTROL_Public","CONTROL_Private_nonprofit","CONTROL_Private for-profit"]
all_features = float_columns.copy()
all_features.extend(catHotColumns)

In [187]:
train_X.columns = all_features

### The following part shows the initial input features preparation
    1. Detecting the Outliers by using skew function
    2. Handling the outliers by replacing the outliers (less than 10% or larger than 90%) with the First and 
    third Quantile

In [188]:
for col in train_X.columns:
    print(col, "\t", train_X[col].skew(), "\n")
    
outliers = ['ACTENMID', 'ACTENMID', 'ACTWRMID', 'SAT_AVG', 'SAT_AVG_ALL', 'SATMTMID', 'UGDS', 'TUITFTE', 'INEXPFTE',
           'COSTT4_P', 'UGDS_BLACK', 'UGDS_HISP', 'UGDS_ASIAN', 'UGDS_AIAN', 'UGDS_NHPI', 'UGDS_2MOR',
           'UGDS_NRA', 'UGDS_UNKN', 'TUITIONFEE_PROG', 'DEATH_YR3_RT', 'LOAN_DEATH_YR3_RT', 'DEATH_YR4_RT',
           'COUNT_NWNE_P10', 'COUNT_WNE_P10', 'HIGHDEG_Non-degree-granting']

ADM_RATE 	 -1.1389354030588208 

ADM_RATE_ALL 	 -1.2364634807491872 

ACTCMMID 	 0.8479968824827924 

ACTENMID 	 1.7674415564711146 

ACTMTMID 	 1.9192349338510628 

ACTWRMID 	 7.398108987200559 

SAT_AVG 	 2.1573139587289214 

SAT_AVG_ALL 	 1.8589879397055689 

SATMTMID 	 2.247876534261437 

UGDS 	 7.9136754866592405 

TUITFTE 	 TUITFTE    3.772172
TUITFTE    3.772172
dtype: float64 

INEXPFTE 	 INEXPFTE    9.949202
INEXPFTE    9.949202
dtype: float64 

COSTT4_P 	 COSTT4_P    2.226925
COSTT4_P    2.226925
dtype: float64 

COSTT4_A 	 COSTT4_A    1.226009
COSTT4_A    1.226009
dtype: float64 

PCTFLOAN 	 -0.7161603961158066 

COMP_ORIG_YR2_RT 	 0.5698450626947548 

UGDS_WHITE 	 -0.36580027282275723 

UGDS_BLACK 	 1.8241999616639633 

UGDS_HISP 	 2.4797150379155597 

UGDS_ASIAN 	 4.922824661132807 

UGDS_AIAN 	 14.580523087121943 

UGDS_NHPI 	 19.6671210505093 

UGDS_2MOR 	 4.277533713367993 

UGDS_NRA 	 5.053697211987563 

UGDS_UNKN 	 3.7428552851693326 

PPTUG_EF 	 1.1327263727462669 



In [189]:
def handleOutliers(outliers):
    for col in outliers:
        Q1 = train_X[col].quantile(0.10)
        Q3 = train_X[col].quantile(0.90)
        #print("col: ", col, "Q1: ", Q1, "Q3: ", Q3)
        
        train_X[col] = np.where(train_X[col] <Q1, Q1,train_X[col])
        train_X[col] = np.where(train_X[col] >Q3, Q3,train_X[col])
        print("New Skew Value: ", train_X[col].skew())

In [190]:
#handleOutliers(outliers)

### Exploring features importance by applying "RFE": Recursive Feature Estimation

In [191]:
from sklearn.feature_selection import RFE
def recursive_feat_estimate(model):
    rfe = RFE(model, 10)
    fit = rfe.fit(train_X, train_y)
    print("Features sorted by their rank:")
    print(sorted(zip(map(lambda x: x, rfe.ranking_), all_features)))
    return rfe

### Fit, Train and Predict the model with initial alpha "1e-3"

In [192]:
#--fit the Ridge model
ridgereg = Ridge(alpha=1e-3,normalize=True)

In [193]:
ridgereg.fit(train_X, train_y)

Ridge(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)

In [194]:
pred = ridgereg.predict(test_X)
trainingPred = ridgereg.predict(train_X)

In [195]:
ridgereg.score(test_X, test_y)

0.7513943905694749

In [196]:
#--Calculate Root mean square error
from sklearn.metrics import mean_squared_error

mean = mean_squared_error(test_y, pred) 
rmse = np.sqrt(mean_squared_error(test_y,pred))

#--Calculate Root mean square error on training set
trainMean = mean_squared_error(train_y, trainingPred) 
trainRmse = np.sqrt(mean_squared_error(train_y,trainingPred)) 

print("Mean: ", mean)
print("RMSE: ", rmse)


print("TrainMean: ", trainMean)
print("TrainRMSE: ", trainRmse)

Mean:  0.009804302916383945
RMSE:  0.09901667999071644
TrainMean:  0.010007694100320057
TrainRMSE:  0.10003846310454823


In [197]:
from sklearn.metrics import r2_score
r2_score(test_y, pred)

0.7513943905694749

In [198]:
r2_score(train_y, trainingPred)

0.7428959237243533

In [199]:
recursive_feat_estimate(ridgereg)

Features sorted by their rank:
[(1, 'ADM_RATE'), (1, 'ADM_RATE_ALL'), (1, 'COMP_ORIG_YR2_RT'), (1, 'COMP_ORIG_YR4_RT'), (1, 'DEATH_YR3_RT'), (1, 'DEATH_YR4_RT'), (1, 'LOAN_DEATH_YR3_RT'), (1, 'PCTPELL'), (1, 'UGDS_BLACK'), (1, 'UGDS_NRA'), (2, 'CONTROL_Private for-profit'), (3, 'UGDS_ASIAN'), (4, 'UGDS_WHITE'), (5, 'PCTFLOAN'), (6, 'COMP_ORIG_YR3_RT'), (7, 'HIGHDEG_Non-degree-granting'), (8, 'LOAN_COMP_ORIG_YR3_RT'), (9, 'CONTROL_Public'), (10, 'CONTROL_Private_nonprofit'), (11, 'UGDS_2MOR'), (12, 'HIGHDEG_Certificate_degree'), (13, 'HIGHDEG_Associate_degree'), (14, 'PPTUG_EF'), (15, 'UGDS_NHPI'), (16, 'HIGHDEG_Associate_degree'), (17, 'UGDS_UNKN'), (18, 'UGDS_HISP'), (19, 'UGDS_AIAN'), (20, 'AGE_ENTRY'), (21, 'ACTMTMID'), (22, 'ACTWRMID'), (23, 'ACTCMMID'), (24, 'HIGHDEG_Associate_degree'), (25, 'ACTENMID'), (26, 'SAT_AVG_ALL'), (27, 'SATMTMID'), (28, 'SAT_AVG'), (29, 'MD_EARN_WNE_P10'), (30, 'AVGFACSAL'), (31, 'COUNT_NWNE_P10'), (32, 'COUNT_WNE_P10'), (33, 'COSTT4_P'), (34, 'TUITIONF

RFE(estimator=Ridge(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001),
  n_features_to_select=10, step=1, verbose=0)

In [200]:
ridgereg.coef_

array([ 2.94734679e-02, -2.38839521e-02, -7.71043939e-03,  9.87080222e-04,
        2.30279466e-03, -9.11832568e-03,  2.86649898e-04,  1.76086646e-04,
       -3.73084213e-04, -8.81616402e-07, -1.76727165e-07,  2.86476892e-07,
       -9.01722667e-07, -9.10916525e-07,  5.51496479e-02, -6.83891563e-02,
        8.59152923e-02, -1.28505896e-01,  5.21505518e-03,  2.03299681e-01,
        3.71378363e-02, -3.53530471e-02, -5.95225520e-02,  1.88097390e-01,
       -1.09125670e-02,  7.04237992e-02, -9.10916525e-07, -9.01722667e-07,
        2.71962131e-06, -2.13426375e-06,  1.04743137e-07, -1.76727165e-07,
        2.86476892e-07, -3.58722666e-06, -1.73142466e-01,  4.01115101e+00,
        8.34887166e-02,  3.12536650e+01, -5.45617642e-02,  2.37375685e+00,
        6.86481066e-02, -1.24159797e-02, -2.66487574e-05,  4.03221711e-06,
        6.18024984e-07,  3.29370568e-06, -4.87340718e-02,  2.33475913e-02,
       -2.37724863e-02, -5.93894490e-03,  2.27289934e-02,  4.53090687e-02,
        3.97583301e-02, -

# Explore Ridge regression with built-in cross-validation.

In [211]:
#--Trying different alphas with CV = 3
a = [1e-5, 1e-4, 1e-3, 0.01, 0.1, 1, 10, 100, 1000, 10000]
ridgeCV = RidgeCV(alphas=a, normalize=True, cv=10, scoring='r2')

In [212]:
ridgeCV.fit(train_X, train_y)
print("Best alpha using built-in RidgeCV: %f" % ridgeCV.alpha_)

Best alpha using built-in RidgeCV: 0.001000


In [213]:
#--using the best alpha to predict the test data 
alpha = ridgeCV.alpha_
ridgeCV = Ridge(alpha=alpha,normalize=True)
ridgeCV.fit(train_X, train_y)

Ridge(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)

In [214]:
predCV = ridgeCV.predict(test_X)
trainingPredCV = ridgeCV.predict(train_X)

In [217]:
#--Calculate Root mean square error
from sklearn.metrics import mean_squared_error

mean = mean_squared_error(test_y, predCV) 
rmse = np.sqrt(mean_squared_error(test_y,predCV))

#--Calculate Root mean square error on training set
trainMean = mean_squared_error(train_y, trainingPredCV) 
trainRmse = np.sqrt(mean_squared_error(train_y,trainingPredCV)) 

print("Mean: ", mean)
print("RMSE: ", rmse)


print("TrainMean: ", trainMean)
print("TrainRMSE: ", trainRmse)

Mean:  0.009804302916383945
RMSE:  0.09901667999071644
TrainMean:  0.010007694100320057
TrainRMSE:  0.10003846310454823


In [218]:
from sklearn.metrics import r2_score
r2_score(test_y, predCV)

0.7513943905694749

In [219]:
r2_score(train_y, trainingPredCV)

0.7428959237243533

In [223]:
ridgeCV.coef_

array([ 2.94734679e-02, -2.38839521e-02, -7.71043939e-03,  9.87080222e-04,
        2.30279466e-03, -9.11832568e-03,  2.86649898e-04,  1.76086646e-04,
       -3.73084213e-04, -8.81616402e-07, -1.76727165e-07,  2.86476892e-07,
       -9.01722667e-07, -9.10916525e-07,  5.51496479e-02, -6.83891563e-02,
        8.59152923e-02, -1.28505896e-01,  5.21505518e-03,  2.03299681e-01,
        3.71378363e-02, -3.53530471e-02, -5.95225520e-02,  1.88097390e-01,
       -1.09125670e-02,  7.04237992e-02, -9.10916525e-07, -9.01722667e-07,
        2.71962131e-06, -2.13426375e-06,  1.04743137e-07, -1.76727165e-07,
        2.86476892e-07, -3.58722666e-06, -1.73142466e-01,  4.01115101e+00,
        8.34887166e-02,  3.12536650e+01, -5.45617642e-02,  2.37375685e+00,
        6.86481066e-02, -1.24159797e-02, -2.66487574e-05,  4.03221711e-06,
        6.18024984e-07,  3.29370568e-06, -4.87340718e-02,  2.33475913e-02,
       -2.37724863e-02, -5.93894490e-03,  2.27289934e-02,  4.53090687e-02,
        3.97583301e-02, -

In [224]:
recursive_feat_estimate(ridgeCV)

Features sorted by their rank:
[(1, 'ADM_RATE'), (1, 'ADM_RATE_ALL'), (1, 'COMP_ORIG_YR2_RT'), (1, 'COMP_ORIG_YR4_RT'), (1, 'DEATH_YR3_RT'), (1, 'DEATH_YR4_RT'), (1, 'LOAN_DEATH_YR3_RT'), (1, 'PCTPELL'), (1, 'UGDS_BLACK'), (1, 'UGDS_NRA'), (2, 'CONTROL_Private for-profit'), (3, 'UGDS_ASIAN'), (4, 'UGDS_WHITE'), (5, 'PCTFLOAN'), (6, 'COMP_ORIG_YR3_RT'), (7, 'HIGHDEG_Non-degree-granting'), (8, 'LOAN_COMP_ORIG_YR3_RT'), (9, 'CONTROL_Public'), (10, 'CONTROL_Private_nonprofit'), (11, 'UGDS_2MOR'), (12, 'HIGHDEG_Certificate_degree'), (13, 'HIGHDEG_Associate_degree'), (14, 'PPTUG_EF'), (15, 'UGDS_NHPI'), (16, 'HIGHDEG_Associate_degree'), (17, 'UGDS_UNKN'), (18, 'UGDS_HISP'), (19, 'UGDS_AIAN'), (20, 'AGE_ENTRY'), (21, 'ACTMTMID'), (22, 'ACTWRMID'), (23, 'ACTCMMID'), (24, 'HIGHDEG_Associate_degree'), (25, 'ACTENMID'), (26, 'SAT_AVG_ALL'), (27, 'SATMTMID'), (28, 'SAT_AVG'), (29, 'MD_EARN_WNE_P10'), (30, 'AVGFACSAL'), (31, 'COUNT_NWNE_P10'), (32, 'COUNT_WNE_P10'), (33, 'COSTT4_P'), (34, 'TUITIONF

RFE(estimator=Ridge(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001),
  n_features_to_select=10, step=1, verbose=0)